In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif, RFE
import shap
from tabpfn import TabPFNClassifier
from tabpfn.constants import ModelVersion
from sklearn.metrics import f1_score, roc_auc_score
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import f1_score, roc_auc_score
from tqdm.notebook import tqdm
import time

import joblib
import matplotlib.pyplot as plt

import huggingface_hub
huggingface_hub.login()

ModuleNotFoundError: No module named 'matplotlib.backends.registry'

In [ ]:
selected_descriptor = pd.read_csv('../data/descriptor_selection.csv')

file_md_list = {}
for column in selected_descriptor.columns:
    filename = column
    selected_columns = selected_descriptor[column].iloc[0:].dropna().tolist()
    if filename and selected_columns:
        file_md_list[filename] = selected_columns

data_dir = os.path.join("..", "data", "preprocessed")
result_dir = os.path.join("..", "result", "FTO_TABPFN")
os.makedirs(result_dir, exist_ok=True)

In [39]:
ratio = '10x'
results = {}

file_name = f'descriptors_filtered_FTO_training_{ratio}_ignore3D_False.csv'
data_path = os.path.join(data_dir, f"filtered_FTO_training_{ratio}_ignore3D_False.csv")

df = pd.read_csv(data_path)

md_cols = file_md_list[file_name]
fp_cols = [f'X{i+1}' for i in range(1024)]
filtered_df = df[['potency'] + fp_cols + md_cols]

X = filtered_df.drop('potency', axis=1)
Y = filtered_df['potency']

X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.1, random_state=42, stratify=Y)

print(f'현재 변수 갯수: {len(X_train.columns)}')
print(f'  - Fingerprint: {len(fp_cols)}개')
print(f'  - Molecular Descriptor: {len(md_cols)}개')

현재 변수 갯수: 1040
  - Fingerprint: 1024개
  - Molecular Descriptor: 16개


# 기존 Pycaret 결과

In [36]:
comparison_list = []

# PyCaret Blend 결과
pycaret_blend_path = f"../result/FTO_MACCS/{ratio}_w3D/{ratio}_blend_final_metrics.csv"
blend_metrics = pd.read_csv(pycaret_blend_path)
comparison_list.append(pd.DataFrame({
    'method': ['Blend (PyCaret)'],
    'f1': [blend_metrics['F1'].iloc[0]],
    'auc': [blend_metrics['AUC'].iloc[0]],
    'n_features': [len(X_train.columns)]
}))
        
# PyCaret 개별 모델 결과
pycaret_summary_path = f"../result/FTO_MACCS/{ratio}_w3D/{ratio}_summary_evaluation.csv"
summary_df = pd.read_csv(pycaret_summary_path)
mean_df = summary_df[summary_df['Type'] == 'Mean'].copy()

for _, row in mean_df.iterrows():
    comparison_list.append(pd.DataFrame({
        'method': [f"{row['Model']} (PyCaret)"],
        'f1': [row['F1']],
        'auc': [row['AUC']],
        'n_features': [len(X_train.columns)]
        }))

final_comparison_df = pd.concat(comparison_list, ignore_index=True)

# TabPFN 모델 실험

In [26]:
import joblib

n_features_list = [50, 100, 200, 300, 400, 500]
feature_methods = ['all', 'random', 'mi', 'pca', 'rfe', 'shap', 'md_only']

results = []
saved_models = {}

model_save_dir = os.path.join(result_dir, 'models')
os.makedirs(model_save_dir, exist_ok=True)

if not os.path.exists(model_save_dir) or not any(f.endswith('.joblib') for f in os.listdir(model_save_dir)):
    for method in feature_methods:
        print(f"\n[{method.upper()} Method]")

        # ── MD only ──────────────────────────────────────────────
        if method == 'md_only':
            filtered_df_md = df[['potency'] + md_cols]
            X_md = filtered_df_md.drop('potency', axis=1)
            Y_md = filtered_df_md['potency']

            X_train_md, X_test_md, y_train_md, y_test_md = train_test_split(
                X_md, Y_md, test_size=0.1, random_state=42, stratify=Y_md
            )

            model = TabPFNClassifier.create_default_for_version(ModelVersion.V2_5)
            model.fit(X_train_md.values, y_train_md.values)

            pred = model.predict(X_test_md.values)
            pred_proba = model.predict_proba(X_test_md.values)

            f1 = f1_score(y_test_md, pred, average='weighted')
            auc = roc_auc_score(y_test_md, pred_proba[:, 1])

            print(f"  N_feat: {X_train_md.shape[1]:3d} (MD 전체) - F1: {f1:.4f}, AUC: {auc:.4f}")

            saved_models['md_only_all'] = {
                'model': model,
                'method': method,
                'n_features': X_train_md.shape[1],
                'selected_features': list(X_train_md.columns),
                'transformer': None,
                'X_test': X_test_md,
                'y_test': y_test_md
            }

            results.append({
                'method': method,
                'n_features': X_train_md.shape[1],
                'f1': f1,
                'auc': auc,
                'model_key': 'md_only_all'
            })

        # ── All features ──────────────────────────────────────────
        elif method == 'all':
            X_train_sel = X_train.copy()
            X_test_sel = X_test.copy()

            model = TabPFNClassifier.create_default_for_version(ModelVersion.V2_5)
            model.fit(X_train_sel.values, y_train.values)

            pred = model.predict(X_test_sel.values)
            pred_proba = model.predict_proba(X_test_sel.values)

            f1 = f1_score(y_test, pred, average='weighted')
            auc = roc_auc_score(y_test, pred_proba[:, 1])

            print(f"  N_feat: {X_train.shape[1]:3d} (전체) - F1: {f1:.4f}, AUC: {auc:.4f}")

            saved_models['all_all'] = {
                'model': model,
                'method': method,
                'n_features': X_train.shape[1],
                'selected_features': list(X_train.columns),
                'transformer': None
            }

            results.append({
                'method': method,
                'n_features': X_train.shape[1],
                'f1': f1,
                'auc': auc,
                'model_key': 'all_all'
            })

        # ── Feature selection methods ─────────────────────────────
        else:
            for n_feat in n_features_list:
                transformer = None
                features = None

                if method == 'random':
                    np.random.seed(42)
                    idx = np.random.choice(X_train.shape[1], n_feat, replace=False)
                    features = X_train.columns[idx].tolist()
                    X_train_sel = X_train[features]
                    X_test_sel = X_test[features]

                elif method == 'mi':
                    mi_scores = mutual_info_classif(X_train, y_train, random_state=42, n_jobs=-1)
                    idx = np.argsort(mi_scores)[-n_feat:]
                    features = X_train.columns[idx].tolist()
                    X_train_sel = X_train[features]
                    X_test_sel = X_test[features]

                elif method == 'pca':
                    pca = PCA(n_components=n_feat, random_state=42)
                    X_train_sel = pca.fit_transform(X_train)
                    X_test_sel = pca.transform(X_test)
                    transformer = pca

                elif method == 'rfe':
                    rf_base = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
                    rfe = RFE(estimator=rf_base, n_features_to_select=n_feat, step=50, verbose=0)
                    rfe.fit(X_train, y_train)
                    features = X_train.columns[rfe.support_].tolist()
                    X_train_sel = X_train[features]
                    X_test_sel = X_test[features]
                    transformer = rfe

                elif method == 'shap':
                    rf_shap = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
                    rf_shap.fit(X_train, y_train)

                    sample_size = min(1000, X_train.shape[0])
                    X_sample = X_train.sample(n=sample_size, random_state=42)

                    explainer = shap.TreeExplainer(rf_shap)
                    shap_values = explainer.shap_values(X_sample)

                    shap_importance = np.abs(shap_values).mean(axis=0).mean(axis=1)
                    idx = np.argsort(shap_importance)[-n_feat:]
                    features = X_train.columns[idx].tolist()
                    X_train_sel = X_train[features]
                    X_test_sel = X_test[features]

                model = TabPFNClassifier.create_default_for_version(ModelVersion.V2_5)
                model.fit(X_train_sel, y_train.values)

                pred = model.predict(X_test_sel)
                pred_proba = model.predict_proba(X_test_sel)

                f1 = f1_score(y_test, pred, average='weighted')
                auc = roc_auc_score(y_test, pred_proba[:, 1])

                model_key = f'{method}_{n_feat}'
                saved_models[model_key] = {
                    'model': model,
                    'method': method,
                    'n_features': n_feat,
                    'selected_features': features,
                    'transformer': transformer
                }

                results.append({
                    'method': method,
                    'n_features': n_feat,
                    'f1': f1,
                    'auc': auc,
                    'model_key': model_key
                })

    results_df = pd.DataFrame(results)

    for model_key, model_info in saved_models.items():
        save_path = os.path.join(model_save_dir, f'{model_key}.joblib')
        joblib.dump(model_info, save_path)
    print(f"\n모델 {len(saved_models)}개 저장 완료: {model_save_dir}")

    results_df.sort_values('auc', ascending=False).to_csv(
        os.path.join(model_save_dir, 'results.csv'), index=False)

else:
    for f in sorted(os.listdir(model_save_dir)):
        if f.endswith('.joblib'):
            model_key = f.replace('.joblib', '')
            saved_models[model_key] = joblib.load(os.path.join(model_save_dir, f))

    results_df = pd.read_csv(os.path.join(model_save_dir, 'results.csv'))

In [37]:
results_df.sort_values('f1',ascending=False)

,method,n_features,f1,auc,model_key
2,pca,50,0.975879,0.989539,pca_50
6,pca,500,0.972714,0.985106,pca_500
1,shap,200,0.972158,0.990603,shap_200
25,random,300,0.971544,0.962589,random_300
7,pca,300,0.968506,0.984752,pca_300
4,pca,400,0.968506,0.986525,pca_400
10,rfe,400,0.968506,0.981915,rfe_400
3,pca,200,0.968506,0.986702,pca_200
15,rfe,200,0.968506,0.975000,rfe_200
12,shap,400,0.968506,0.976596,shap_400


## Best model SHAP 분석

In [27]:
for best_model_key in ['pca_50', 'shap_200', 'md_only_all','all_all']:

    # ── 모델 로드 ──────────────────────────────────────────────
    shap_dir = os.path.join(result_dir, 'SHAP')
    os.makedirs(shap_dir, exist_ok=True)
    model_info = joblib.load(os.path.join(model_save_dir, f'{best_model_key}.joblib'))

    model = model_info['model']
    method = model_info['method']
    transformer = model_info['transformer']
    selected_features = model_info['selected_features']

    # ── 데이터/feature 준비 ────────────────────────────────────
    if method == 'md_only':
        X_tr = model_info['X_test'].values  # md_only는 별도 저장
        X_te = model_info['X_test'].values
        y_te = model_info['y_test']
        feature_names = model_info['selected_features']
    elif method == 'pca':
        X_tr = transformer.transform(X_train)
        X_te = transformer.transform(X_test)
        y_te = y_test
        n_feat = model_info['n_features']
        feature_names = [f'PC{i+1}' for i in range(n_feat)]
    elif method == 'rfe':
        X_tr = X_train[selected_features].values
        X_te = X_test[selected_features].values
        y_te = y_test
        feature_names = selected_features
    else:  # all, random, mi, shap
        X_tr = X_train[selected_features].values if selected_features else X_train.values
        X_te = X_test[selected_features].values if selected_features else X_test.values
        y_te = y_test
        feature_names = selected_features if selected_features else X_train.columns.tolist()

    # ── 성능 확인 ──────────────────────────────────────────────
    pred = model.predict(X_te)
    pred_proba = model.predict_proba(X_te)
    f1 = f1_score(y_te, pred, average='weighted')
    auc = roc_auc_score(y_te, pred_proba[:, 1])
    print(f"[{best_model_key}] F1: {f1:.4f}, AUC: {auc:.4f}")

    # ── SHAP 계산 ──────────────────────────────────────────────
    def model_predict(X):
        return model.predict_proba(X)

    X_background = X_tr[:min(50, X_tr.shape[0])]
    print(f"Background: {len(X_background)}개, 샘플: {X_te.shape[0]}개")

    explainer = shap.KernelExplainer(model_predict, X_background)
    shap_values = explainer.shap_values(X_te, nsamples=100)

    sv = np.array(shap_values)
    if sv.ndim == 3:
        shap_values_class1 = sv[:, :, 1]
    else:
        shap_values_class1 = sv[1]

    # ── 시각화 ────────────────────────────────────────────────
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values_class1, X_te, feature_names=feature_names, show=False, max_display=20)
    plt.tight_layout()
    plt.savefig(f'{shap_dir}/shap_{best_model_key}_summary.png', dpi=300, bbox_inches='tight')
    plt.close()

    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values_class1, X_te, feature_names=feature_names, plot_type='bar', show=False, max_display=20)
    plt.tight_layout()
    plt.savefig(f'{shap_dir}/shap_{best_model_key}_bar.png', dpi=300, bbox_inches='tight')
    plt.close()

    # ── Feature Importance 저장 ────────────────────────────────
    pd.DataFrame({
        'feature': feature_names,
        'mean_abs_shap': np.abs(shap_values_class1).mean(axis=0)
    }).sort_values('mean_abs_shap', ascending=False).to_csv(
        f'{shap_dir}/shap_{best_model_key}_importance.csv', index=False)

    # ── PCA일 경우 components 추가 저장 ───────────────────────
    if method == 'pca':
        pd.DataFrame(
            transformer.components_,
            columns=X_train.columns,
            index=feature_names
        ).to_csv(f'{shap_dir}/pca_components_{best_model_key}.csv')

[pca_50] F1: 0.9759, AUC: 0.9895
Background: 50개, 샘플: 259개


  0%|          | 0/259 [00:00<?, ?it/s]

[shap_200] F1: 0.9722, AUC: 0.9906
Background: 50개, 샘플: 259개


  0%|          | 0/259 [00:00<?, ?it/s]

[md_only_all] F1: 0.9671, AUC: 0.9560
Background: 50개, 샘플: 259개


  0%|          | 0/259 [00:00<?, ?it/s]

[all_all] F1: 0.9642, AUC: 0.9761
Background: 50개, 샘플: 259개


  0%|          | 0/259 [00:00<?, ?it/s]

## 생성된 모델들로 예측하기

In [28]:
def predict_with_model_batched(model_key, new_data, training_columns, batch_size=500, verbose=False):
    model_info = saved_models[model_key]
    model = model_info['model']
    method = model_info['method']
    selected_features = model_info['selected_features']
    transformer = model_info['transformer']
    
    new_data_features = new_data[training_columns]
    
    n_samples = len(new_data_features)
    n_batches = (n_samples + batch_size - 1) // batch_size
    
    all_preds = []
    all_probas = []
    
    # 배치별 예측
    batch_iter = range(n_batches)
    if verbose:
        batch_iter = tqdm(batch_iter, desc=f"  {model_key}", leave=False)
    
    for i in batch_iter:
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, n_samples)
        batch_data = new_data_features.iloc[start_idx:end_idx]
        
        # Feature 변환
        if method == 'pca':
            batch_transformed = transformer.transform(batch_data)
        elif method == 'rfe':
            batch_transformed = batch_data[selected_features].values
        else:
            batch_transformed = batch_data[selected_features].values
        
        # 예측
        pred = model.predict(batch_transformed)
        pred_proba = model.predict_proba(batch_transformed)
        
        all_preds.append(pred)
        all_probas.append(pred_proba)
    
    # 결과 합치기
    final_pred = np.concatenate(all_preds)
    final_proba = np.vstack(all_probas)
    
    predictions = pd.DataFrame({
        'prediction': final_pred,
        'probability_class_0': final_proba[:, 0],
        'probability_class_1': final_proba[:, 1]
    })
    
    return predictions

In [29]:
ratio = '10x'
foodb_df = pd.read_csv(f"../data/foodb/filtered_foodb_{ratio}.csv")

training_columns = X_train.columns.tolist()

In [ ]:
BATCH_SIZE = 500
output_dir = '../result/FTO_TABPFN'
os.makedirs(output_dir, exist_ok=True)

existing_files = set()
for f in os.listdir(output_dir):
    if f.endswith('_foodb_predictions.csv'):
        model_key = f.replace('_foodb_predictions.csv', '')
        existing_files.add(model_key)

models_to_predict = {k: v for k, v in saved_models.items() if k not in existing_files}
all_sheets = {}
total_start = time.time()

for model_key in tqdm(models_to_predict.keys(), desc="전체 진행"):

    predictions = predict_with_model_batched(
        model_key, foodb_df, training_columns,
        batch_size=BATCH_SIZE, verbose=False
    )

    class1_mask = predictions['prediction'] == 1

    if class1_mask.sum() > 0:
        result_df = pd.DataFrame({
            'id': foodb_df.loc[class1_mask, 'id'].values,
            'canonical_SMILES': foodb_df.loc[class1_mask, 'canonical_SMILES'].values,
            'prediction': predictions.loc[class1_mask, 'prediction'].values,
            'probability': predictions.loc[class1_mask, 'probability_class_1'].values
        }).sort_values('probability', ascending=False).drop_duplicates('canonical_SMILES')
    else:
        result_df = pd.DataFrame(columns=['id', 'canonical_SMILES', 'prediction', 'probability'])

    csv_path = f'{output_dir}/{model_key}_foodb_predictions.csv'
    result_df.to_csv(csv_path, index=False)

    all_sheets[model_key] = result_df

In [ ]:
excel_path = f'{output_dir}/all_foodb_predictions.xlsx'
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    for model_key, df in all_sheets.items():
        df.to_excel(writer, sheet_name=model_key[:31], index=False)